### 변수

In [45]:
text_path = "/content/drive/MyDrive/Colab Notebooks/milk"
openApiKey = "BD099954D410A11253DA35DEB404198B"
conv_length = 4

### Enviornment Setting

In [2]:
pip install fasttext

In [54]:
!pip install konlpy
%env JAVA_HOME "C:\Program Files\Java\jdk-14.0.2"

     |████████████████████████████████| 19.4 MB 7.1 MB/s 
     |████████████████████████████████| 448 kB 9.3 MB/s 
     |████████████████████████████████| 86 kB 4.4 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


env: JAVA_HOME="C:\Program Files\Java\jdk-14.0.2"


In [107]:
import fasttext
import konlpy
from konlpy.tag import Okt
from konlpy.tag import Komoran
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
import json
import os
import re
import numpy as np
import requests
from bs4 import BeautifulSoup
import random

In [4]:
import fasttext.util
fasttext.util.download_model('ko', if_exists='ignore')

'cc.ko.300.bin'

In [5]:
model = fasttext.load_model('cc.ko.300.bin')

In [66]:
komoran = Komoran()

In [68]:
re = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]")

### util

In [108]:
def search_word(word):

    params = f"?certkey_no=3114&key={openApiKey}&target_type=search&q={word}"
    openUrl = "https://opendict.korean.go.kr/api/search"+params

    paramsForExample = f"?certkey_no=3114&key={openApiKey}&target_type=search&q={word}&part=exam"
    openUrlForExample = "https://opendict.korean.go.kr/api/search"+paramsForExample

    res = requests.get(openUrl)
    soup = BeautifulSoup(res.content, 'html.parser')

    resEx = requests.get(openUrlForExample)
    soupEx = BeautifulSoup(resEx.content, 'html.parser')

    ret_json = {"word":word, "items":[]}
    
    ## 명사 혹은 대명사면 그대로
    if komoran.pos(word)[0][1] == 'NN' or 'NP':
            word = komoran.pos(c)[0][0]

    check = soup.find('total').get_text()
    if check == '0':
        return ret_json

    wordPos = soup.find_all('pos')
    wordDef = soup.find_all('definition')
    wordEx = soupEx.find_all('example')

    for i in range(min(3, len(wordPos))):
        item = {}
        item["pos"] = wordPos[i].get_text().strip()
        item["definition"] = wordDef[i].get_text().strip()
        ret_json["items"].append(item)

    return ret_json

In [50]:
def cos(A, B):
    return (A.dot(B.T)/(np.linalg.norm(A) * np.linalg.norm(B)))

In [96]:
def create_answer(problem):
    sentence_vector = model.get_sentence_vector(problem)
    result = []
    problem = problem.replace("?", " ")
    problem = problem.replace(".", " ")
    problem = problem.replace("!", " ")
    problem = problem.replace("~", " ")
    
    for word in problem.split(" "):
        word_vector = model.get_word_vector(word)
        result.append([cos(sentence_vector, word_vector), word])

    result = sorted(result, key=lambda x: -x[0])
    answer = result[0][1]
    return answer

In [122]:
def create_candidate(answer):
    answer_morphs = komoran.morphs(answer)
    candidate = model.get_nearest_neighbors(answer)
    
    print(candidate)
    candidate = filter(lambda x : re.search(x[1]) == None, candidate)
    candidate = filter(lambda x : komoran.morphs(x[1]) != answer_morphs, candidate)
    candidate = map(lambda x : x[1], candidate)
    candidate = list(candidate)

    print(candidate[:3])

    if len(candidate) < 3:
        return None
    else:
        return candidate[:3]


In [120]:
def change_speaker(context):
    speaker =['승열', '돌맹', '붕어', '감자']
    random.shuffle(speaker)
    s = []

    for c in context:
        if c["speaker"] not in s:
            s.append(c["speaker"])
        c["speaker"] = speaker[s.index(c["speaker"])]

## quiz generator

In [123]:
with open("/content/drive/MyDrive/Colab Notebooks/milk/out_SDRW2000000026.json") as dataset:
    dataset = json.load(dataset)['sentences']

##for index in range(0, len(dataset), conv_length):
for index in range(0, 3, conv_length):

    if index + conv_length > len(dataset):
        break

    ## init
    problem_commit = {
        "context": [],
        "options": [],
        "solution":0
    }
    problem = ""
    context = []
    option = []

    ## problem
    for conv_index in range(conv_length):
        problem += dataset[index + conv_index]['content']
        context.append({
            "content": dataset[index + conv_index]['content'],
            "speaker": dataset[index + conv_index]['speaker']
        })
    context = change_speaker(context)
    
    ## answer
    print(problem)
    answer = create_answer(problem)
    candidate = create_candidate(answer)

    if candidate == None:
        continue
    
    candidate.append(answer)
    random.shuffle(candidate)

    ## answer sheet
    for c in candidate:
        option.append(search_word(c))


    problem_commit["context"] = context
    problem_commit["options"] = option
    problem_commit["solution"] = candidate.index(answer)

    print(problem_commit)



[{'content': '혹시 그동안 해보신 알바들 어떤 게 있으 있는지 말씀해 주시겠어요?', 'speaker': '돌맹'}, {'content': '저는 유니클로 알바랑 미스터피자 그리고 전시회 아르바이트해 본 적 있습니다.', 'speaker': '승열'}, {'content': '어떤 거 해보셨어요?', 'speaker': '승열'}, {'content': '저는 단기 알바들 주로 많이 했었고 일단은 장기적으로 그렇게 장기적인 건 아니었지만 이제 밥버거가 한창 유행할 때 밥버거 집에서 일했었고 최근에는 빵집 파리바게뜨에서 근무를 했었어요.', 'speaker': '돌맹'}]
혹시 그동안 해보신 알바들 어떤 게 있으 있는지 말씀해 주시겠어요?저는 유니클로 알바랑 미스터피자 그리고 전시회 아르바이트해 본 적 있습니다.어떤 거 해보셨어요?저는 단기 알바들 주로 많이 했었고 일단은 장기적으로 그렇게 장기적인 건 아니었지만 이제 밥버거가 한창 유행할 때 밥버거 집에서 일했었고 최근에는 빵집 파리바게뜨에서 근무를 했었어요.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in float_scalars
  


[(0.7467092275619507, '제가'), (0.7320271730422974, '저도'), (0.6367291808128357, '저의'), (0.5821576714515686, '아뇨'), (0.580600917339325, '저와'), (0.5532360672950745, '하핫'), (0.5427647829055786, '제겐'), (0.5355950593948364, '근데'), (0.5323740839958191, '알기로도'), (0.5310829877853394, '저역시')]


TypeError: ignored

In [ ]:
problems = []
for file_name in os.listdir(text_path):

    with open(text_path + file_name) as dataset:
        dataset = json.load(dataset)['sentences']

    for index in range(0, len(dataset), 3):

        if index + 2 > len(dataset) - 1:
            break
        problem_commit = {
            "problem": None,
            "solution": None,
        }

        problem = dataset[index]['content'] + dataset[index +
                                                      1]['content'] + dataset[index + 2]['content']

        problem_commit["problem"] = {"problem": [
            {
                "content": dataset[index]['content'],
                "speaker": dataset[index]['speaker']
            },
            {
                "content": dataset[index + 1]['content'],
                "speaker": dataset[index + 1]['speaker']
            },
            {
                "content": dataset[index + 2]['content'],
                "speaker": dataset[index + 2]['speaker']
            },
        ]}
        sentence_vector = model.get_sentence_vector(problem)
        result = []
        problem = problem.replace("?", " ")
        problem = problem.replace(".", " ")
        problem = problem.replace("!", " ")
        problem = problem.replace("~", " ")
        for word in problem.split(" "):
            word_vector = model.get_word_vector(word)
            result.append([cos(sentence_vector, word_vector), word])

        result = sorted(result, key=lambda x: -x[0])
        problem_commit['solution'] = result[0][1]
